# Model Training w/ Patient List

In this notebook, we'll demonstrate how to generate CLMBR features given a list of patients and labels using a pre-trained CLMBR model, and train a simple classifier on top of a CLMBR-featurized dataset. The example dataset used here is a STARR OMOP dataset.

In [ ]:
import torch

In [ ]:
import os
import json
import ehr_ml.timeline
import ehr_ml.ontology
import ehr_ml.index
import ehr_ml.labeler
import ehr_ml.clmbr
import pandas as pd

from path_utils import load_extract_paths, update_extract_paths, load_dataset_paths, update_dataset_paths

MACHINE = 'nero'
DATASET = 'starr_omop_deid'
VERSION = '2021_12_13'
#paths = load_dataset_paths(MACHINE, DATASET, VERSION)
EHR_ML_EXTRACT_DIR = '/home/kbechler/dataset'
#paths.get('extract_dir', None)
CLMBR_INFO_DIR = "/invalid" 
#paths.get('info_dir', None)
MODEL_DIR = '/home/kbechler/model2'
#paths.get('model_dir', None)
#EXAMPLE_CSV = paths.get('example_csv', None)

# make any updates to the paths dictionary, and update the persistent config file with:
# update_dataset_paths(MACHINE, DATASET, VERSION, paths)

In [ ]:
print(EHR_ML_EXTRACT_DIR)

In [ ]:
print(MODEL_DIR)

### Featurizing from a list of patients
In this use case, we assume that the user has prepared a list of patients, day offsets, and labels for use with training the model. Such a list can come from a query to BigQuery or similar service. We'll load in an example CSV file to demonstrate the minimum spec required.

In [ ]:
os.getcwd()

In [ ]:
# Replace with SLE dataframe of patient list
dataframe = pd.read_csv('/home/kbechler/piton_private/df_clmbr_2022.csv')

In [ ]:
dataframe.head()

In [ ]:
# Remove patients less than 14 years old
more_14 = pd.read_csv('/home/kbechler/piton_private/patient_df_less_14.csv')

In [ ]:
patient_14 = list(more_14.subject_id)

In [ ]:
dataframe = dataframe[dataframe.patient_id.isin(patient_14)]

In [ ]:
sum(dataframe.label == False)

In [ ]:
sum(dataframe.label == True)

In [ ]:
print(len(dataframe))

In [ ]:
# Remove held out test set
# Import held out test set to remove patients
heldout_test_set = pd.read_csv('/home/kbechler/test_set.csv')

In [ ]:
test_patients = list(heldout_test_set.patient_id)

In [ ]:
dataframe_train = dataframe[~dataframe.patient_id.isin(test_patients)]

In [ ]:
print(len(dataframe_train))

In [ ]:
dataframe_test = dataframe[dataframe.patient_id.isin(test_patients)]

In [ ]:
print(len(dataframe_test))

We note that if the data was the direct result of a BigQuery query, the patient IDs here don't exactly correspond to the patient IDs used by the `ehr_ml` Timeline object. Furthermore, we need to convert the date strings to date indices to index the last date we should use to featurize each patient. We'll do this preprocessing below using `ehr_ml.clmbr.convert_patient_data`.

In [ ]:
ehr_ml_patient_ids, day_indices = ehr_ml.clmbr.convert_patient_data(EHR_ML_EXTRACT_DIR, 
                                                                    dataframe_train['patient_id'], dataframe_train['date'])

With the correct patient IDs and day indices in hand, we can now generate patient features using our pre-trained model. This is done in two steps:
1. Load in the pre-trained model with `ehr_ml.clmbr.CLMBRFeaturizer.from_pretrained`
2. Featurize with the `featurize_patients` method (**NOTE**: this method expects the _converted_ patient IDs and day indices as arguments!)

In [ ]:
import numpy as np
ehr_ml_patient_ids = np.array(ehr_ml_patient_ids)
day_indices = np.array(day_indices)
labels = dataframe_train['label'].to_numpy()
clmbr_model = ehr_ml.clmbr.CLMBR.from_pretrained(MODEL_DIR)
features = clmbr_model.featurize_patients(EHR_ML_EXTRACT_DIR, ehr_ml_patient_ids, day_indices)

The tensors of interest for training our machine learning model are `features` and `labels`, which define our patient feature-matrix and our task-specific labels respectively.

In [ ]:
features.shape, labels.shape

### Training a logistic regression model
Next, we'll train a logistic regression model which can perform predictions based off our CLMBR representations. We'll first build a simple dataset out of our features and labels, and then define the model. We'll define a simple train / test split so we can measure the performance of our model.

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
# split patients into train / test cohorts, measure accuracy
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

BATCH_SIZE = 1024
EPOCHS = 100
EARLY_STOPPING_EPOCHS = 5
seed = 10

X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size = 0.8, random_state = seed)

model = LogisticRegressionCV(Cs = 10**(np.linspace(-8,8,20)), scoring = "roc_auc").fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
import sklearn

In [ ]:
y_predict = model.predict_log_proba(X_test)

In [ ]:
sklearn.metrics.roc_auc_score(y_test, y_predict[:,1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn import metrics

In [ ]:
results = {}

In [ ]:
metric = ["accuracy", "balanced_accuracy", "roc_auc", "average_precision", "f1", "precision", "recall"]
def validation_results(clf, X_val, y_val, results):
    y_pred = clf.predict(X_val)
    y_score = clf.predict_proba(X_val)[:, 1]
    
    acc = metrics.accuracy_score(y_val, y_pred)
    bal_acc = metrics.balanced_accuracy_score(y_val, y_pred)
    roc_auc = metrics.roc_auc_score(y_val, y_score)
    avg_prec = metrics.average_precision_score(y_val, y_score)
    f1 = metrics.f1_score(y_val, y_pred)
    precision = metrics.precision_score(y_val, y_pred)
    recall = metrics.recall_score(y_val, y_pred)     
    results['CLMBR'] = {"accuracy": acc,
                     "balanced_accuracy": bal_acc,
                     "roc_auc": roc_auc,
                     "average_precision": avg_prec,
                     "f1": f1,
                     "precision": precision,
                     "recall": recall}
    
    return results

In [ ]:
# Plot roc pr curves
def plot_roc_pr_curves(X, y, clf, fig_size=(10, 8)):
    fig, ax = plt.subplots(1,1,figsize=fig_size)
    roc_disp = metrics.plot_roc_curve(clf, X, y, ax=ax)
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title("Receiver Operating Characteristic (ROC) Curve")
    plt.savefig('/home/kbechler/piton_private/plots/model_val_roc.png')
    plt.show()

    # PR Curve and AP (average precision)
    fig, ax = plt.subplots(1,1,figsize=fig_size)
    pr_disp = metrics.plot_precision_recall_curve(clf, X, y, ax=ax)
    ax.set_xlabel("Recall")
    ax.set_ylabel("Precision")
    ax.set_title("Precision-Recall (PR) Curve")
    plt.savefig('/home/kbechler/piton_private/plots/model_val_pr.png')
    plt.show()


In [ ]:
results = validation_results(model, X_test, y_test, results)

In [ ]:
results

In [ ]:
results_df = pd.DataFrame.from_dict(results, orient = 'index', columns = ['accuracy', 'balanced_accuracy', 
                                                                                   'roc_auc', 'average_precision', 
                                                                                   'f1', 'precision', 'recall'])

In [ ]:
results_df.to_csv('/home/kbechler/piton_private/plots/results.csv')

In [ ]:
plot_roc_pr_curves(X_test, y_test, model)
#plt.savefig('/home/kbechler/piton_private/plots/model_val_auc.png')

### Using the trained model

Next we'll show how we can use our simple linear classifier to perform predictions on patients in new extracts. To do so, we need to provide a list of patient IDs and day offsets from a dataset extract to featurize them. For our purposes, we'll re-use a small list we got from our labeller, but you can imagine that these patient IDs can come from some other source (e.g. a query to BigQuery; in that case, we would have to perform our conversion to `ehr_ml`-friendly patient IDs and day indices again).

In [ ]:
ehr_ml_patient_ids, day_indices = ehr_ml.clmbr.convert_patient_data(EHR_ML_EXTRACT_DIR, 
                                                                    dataframe_test['patient_id'], 
                                                                    dataframe_test['date'])

In [ ]:
import numpy as np
ehr_ml_patient_ids = np.array(ehr_ml_patient_ids)
day_indices = np.array(day_indices)
labels = dataframe_test['label'].to_numpy()
clmbr_model = ehr_ml.clmbr.CLMBR.from_pretrained(MODEL_DIR)
features = clmbr_model.featurize_patients(EHR_ML_EXTRACT_DIR, ehr_ml_patient_ids, day_indices)

In [ ]:
# SUBSET_SIZE = 1000

# pid_subset = ehr_ml_patient_ids[:SUBSET_SIZE]
# day_indices_subset = day_indices[:SUBSET_SIZE]

In [ ]:
features.shape, labels.shape

As before, we'll generate the features with `featurize_patients`. Note that while we use the same extract dir in this particular example, you will need to change out the extract dir to test on a different dataset.

In [ ]:
# features = clmbr_model.featurize_patients(EHR_ML_EXTRACT_DIR, pid_subset, day_indices_subset)
# features.shape

In [ ]:
predictions = model.predict(features)
#predictions

In [ ]:
predictions_proba = model.predict_log_proba(features)

In [ ]:
sklearn.metrics.roc_auc_score(labels, predictions_proba[:,1])

In [ ]:
def test_model(clf, X_test, y_test, fig_size=(10,8)):
    # clf: trained classifier (i.e. after using fit function)
    y_pred = clf.predict(X_test)
    y_pred_proba = clf.predict_proba(X_test)

    # print out stats
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Model accuracy: %.3f\n" % accuracy)

    # precision, recall, and f1-score is usually reported for class 1 (in binary case)
    # recall of positive class (1) = sensitivity
    # recall of negative class (0) = specificity
    # precision of positive class (1) = PPV
    # precision of negative class (0) = NPV
    print(metrics.classification_report(y_test,y_pred))

    # confusion matrix
    cm_disp = metrics.plot_confusion_matrix(clf, X_test, y_test)  
    plt.savefig('/home/kbechler/piton_private/plots/model_test_cm.png')
    plt.show()

    # ROC Curve and AUC
    auroc = metrics.roc_auc_score(y_test, y_pred_proba[:, 1])
    fig_roc, ax_roc = plt.subplots(1,1,figsize=fig_size)
    roc_disp = metrics.plot_roc_curve(clf, X_test, y_test, ax=ax_roc)
    ax_roc.set_xlabel("False Positive Rate")
    ax_roc.set_ylabel("True Positive Rate")
    ax_roc.set_title("Receiver Operating Characteristic (ROC) Curve")
    plt.savefig('/home/kbechler/piton_private/plots/model_test_roc.png')
    plt.show()

    # PR Curve and AP (average precision)
    fig, ax_pr = plt.subplots(1,1,figsize=fig_size)
    pr_disp = metrics.plot_precision_recall_curve(clf, X_test, y_test, ax=ax_pr)
    ax_pr.set_xlabel("Recall")
    ax_pr.set_ylabel("Precision")
    ax_pr.set_title("Precision-Recall (PR) Curve")
    plt.savefig('/home/kbechler/piton_private/plots/model_test_pr.png')
    plt.show()

In [ ]:
features.shape

In [ ]:
labels.shape

In [ ]:
os.getcwd()

In [ ]:
test_model(model, features, labels)

In [ ]:
test_results = {}
test_results = validation_results(model, features, labels, test_results)

In [ ]:
test_results

In [ ]:
test_results_df = pd.DataFrame.from_dict(test_results, orient = 'index', columns = ['accuracy', 'balanced_accuracy', 
                                                                                   'roc_auc', 'average_precision', 
                                                                                   'f1', 'precision', 'recall'])

In [ ]:
test_results_df.to_csv('/home/kbechler/piton_private/plots/test_results.csv')

In [ ]:
# Plot both validation and test roc on same plot

def plot_both(clf, X_val, y_val, X_test, y_test, fig_size=(10,8)):
    # clf: trained classifier (i.e. after using fit function)
    y_pred = clf.predict(X_val)
    y_pred_proba = clf.predict_proba(X_val)


    # ROC Curve and AUC
    auroc = metrics.roc_auc_score(y_val, y_pred_proba[:, 1])
    fig_roc, ax_roc = plt.subplots(1,1,figsize=fig_size)
    roc_disp = metrics.plot_roc_curve(clf, X_val, y_val, ax=ax_roc, name = "Internal Validation")
    ax_roc.set_xlabel("False Positive Rate")
    ax_roc.set_ylabel("True Positive Rate")
    ax_roc.set_title("Receiver Operating Characteristic (ROC) Curve")
    
    y_pred_test = clf.predict(X_test)
    y_pred_proba_test = clf.predict_proba(X_test)


    auroc_test = metrics.roc_auc_score(y_test, y_pred_proba_test[:, 1])

    roc_disp = metrics.plot_roc_curve(clf, X_test, y_test, ax=ax_roc, name = "Internal Evaluation on Held-Out Test Set")
    plt.savefig('/home/kbechler/piton_private/plots/test_val_roc.png')
    
    plt.show()

In [ ]:
plot_both(model, X_test, y_test, features, labels)
